In [ ]:
#using DifferentialEquations
#using ADCME
using Plots

In [ ]:
function f_hi(a, T, R)
        v1 = (a[:,2] + a[:,3] * T + a[:,4] * T^2 + a[:,5] * T^3 + a[:,6] * T^4) * R
        v2 = (a[:,2] + a[:,3] * T / 2 + a[:,4] * T^2 /3 + a[:,5] * T^3 /4 + a[:,6] * T^4 /5 + a[:,7] / T) * R * T
        v3 = (a[:,2] * log(T) + a[:,3] * T + a[:,4] * T^2 /2 + a[:,5] * T^3 /3 + a[:,6] * T^4 / 4 + a[:,8]) * R
    return [v1 v2 v3]
end
 
function f_lo(a, T, R)
    v1 = (a[:,9] + a[:,10] * T + a[:,11] * T^2 + a[:,12] * T^3 + a[:,13] * T^4) * R
    v2 = (a[:,9] + a[:,10] * T / 2 + a[:,11] * T^2 /3 + a[:,12] * T^3 /4 + a[:,13] * T^4 /5 + a[:,14]/T) * R * T
    v3 = (a[:,9] * log(T) + a[:,10] * T + a[:,11] * T^2 /2 + a[:,12] * T^3 /3 + a[:,13] * T^4 / 4 + a[:,15]) * R
    return [v1 v2 v3]
end 

function compute_falloff(T, pr, a)
    fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
    c = -0.4 - 0.67 * log(10, fcent)
    n = 0.75 - 1.27 * log(10, fcent)
    f1 = (log(10, pr) + c) / (n - 0.14 *(log(10, pr) + c))
    return  10 ^ (log(10, fcent) / (1 + f1 ^ 2))
end

In [ ]:
using PyCall
#### Read data
py"""
import numpy as np
"""

In [ ]:
length_chamber = 20e-2
# [m]
radius_chamber = 4e-2
# Volume chamber [m^3]
const V_ch = 2 * π * radius_chamber  
# [m]
radius_nozzle = 0.025 
# Nozzle throat nozzle area [m^2]
const A_nozzle = 2 * π * radius_nozzle 
#V = V_ch # Volume
V = 7.577283268106298e-08
#m_ = (py"np.load"("data/density.npy"))[1] # Total mass
m_ = 2.117055938427438e-08
tbd = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/tbd.npy") .+ 1
tbd = Int.(tbd)
falofr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/falofr.npy") .+ 1
falofr = Int.(falofr)
elmr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/elmr.npy") .+ 1
elmr = Int.(elmr)
NASA_coeffs = py"np.load"("data/NASA_coeffs.npy")
W = py"np.load"("data/molecular_weights.npy") # Molar weight
ν1 = py"np.load"("data/reactants_stoich_coeffs.npy") # Forward molar stoichiometric coefficients
ν2 = py"np.load"("data/product_stoich_coeffs.npy") # Backward model stoichiometric coefficients
reversible = py"np.load"("data/reversible.npy")
N = size(ν1)[1]  # Number of Species
M = size(ν1)[2]  # Number of Reactions
ν1_order = zeros(N,M)
ν = ν2 - ν1  
pa = 100000 # 1 bar
R = 8314.4621 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 1000.0
r_o = 7.74132634e-01
Yin = [r_o; 0; 1 - r_o; 0; 0; 0] # Mass fraction of species entering the chamber
Yout =[0; 0.0; 1.0; 0.0; 0.0; 0.0] # Mass fraction of species leaving the chamber
#Yout = reshape(Yout, (6,1))
Qdot = 0 # Heating source
### Unknowns
#Tt_cant = py"np.load"("data/temperature.npy")
#T = Tt_cant[1] # Temperature
#Y = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/ini_mass_fraction.npy") # Mass fractions
Y = [8.04293023e-04; 0.0; 9.99195707e-01; 0.0; 0.0; 0.0]
### h_in
T_init = 350
islarge = NASA_coeffs[:,1] .< T_init * ones(N)
fhi = f_hi(NASA_coeffs, T_init, R)
flo = f_lo(NASA_coeffs, T_init, R)
hin = islarge .* fhi[:,2] + (1 .- islarge) .* flo[:,2]
hin = sum(hin ./ W .* Yin)
### Quantities that depend on the Unknowns
order = py"np.load"("data/reaction_orders.npy")
Af = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor.npy") # preexponential constant Afj
β = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent.npy") # Temperature exponent
E = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy.npy") # Activation energy for the reactions in kJ

order_t = py"np.load"("data/reaction_orders_t.npy")
efficiency_t = py"np.load"("data/efficiency_t.npy")
Af_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_t.npy") # preexponential constant Afj
β_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_t.npy") # Temperature exponent
E_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_t.npy") # Activation energy for the reactions in kJ

order_f = py"np.load"("data/reaction_orders_f.npy")
troefall = py"np.load"("data/troefall.npy") .+ 1
troefall_coeff = py"np.load"("data/troefall_coeff.npy")
efficiency_f = py"np.load"("data/efficiency_f.npy")
Af_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_hi.npy") # preexponential constant Afj
β_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_hi.npy") # Temperature exponent
E_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_hi.npy") 

Af_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_lo.npy") # preexponential constant Afj
β_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_lo.npy") # Temperature exponent
E_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_lo.npy") 

ν1_order[:,elmr] = order
ν1_order[:,tbd] = order_t
ν1_order[:,falofr] = order_f


function q(t)
    t0 = 1e-6
    r = 6e-8
    c = 0
    return c * exp(-(t-t0)^2 * 4 * log(2) / r^2)
end

function f(mTY, t, min, mout)
    m = mTY[1]
    T = mTY[2]
    Y = mTY[3:end]
    ρ = m / V # density
    X = ρ * Y ./ W # Concentration
    n_ = sum(X) * V
    Q = ones(M) # Individual progress rates
    islarge = NASA_coeffs[:,1] .< T * ones(N)
    fhi = f_hi(NASA_coeffs, T, R)
    flo = f_lo(NASA_coeffs, T, R)
    cp = islarge .* fhi[:,1] + (1 .- islarge) .* flo[:,1]
    h = islarge .* fhi[:,2] + (1 .- islarge) .* flo[:,2]
    s = islarge .* fhi[:,3] + (1 .- islarge) .* flo[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    ####
    M_t = efficiency_t' * X
    Kf_t = Af_t .* (T .^ β_t) .* exp.(-E_t ./ (R * T)) .* M_t
    Kr_t = Kf_t ./ (((pa/(R * T)) .^ sum(ν[:,tbd], dims=1)' .* exp.(ΔS[tbd] ./ R - ΔH[tbd] ./ (R * T))))
    Qtbd = Kf_t .* prod(X .^ order_t, dims=1)' .- Kr_t .* prod(X .^ ν2[:,tbd], dims=1)' .* reversible[tbd]
    ####
    M_f = efficiency_f' * X
    Kf_lo = Af_lo .* (T .^ β_lo) .* exp.(-E_lo ./ (R * T)) .* M_f
    Kf_hi = Af_hi .* (T .^ β_hi) .* exp.(-E_hi ./ (R * T)) 
    Pr = Kf_lo ./ Kf_hi
    Fac = ones(size(falofr)[1])
    Fac[troefall] = [compute_falloff(T, Pr[s], troefall_coeff[:,i]) for (i,s) in enumerate(troefall)]
    Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
    Kr_f = Kf_f ./ (((pa/(R * T)) .^ sum(ν[:,falofr], dims=1)' .* exp.(ΔS[falofr] ./ R - ΔH[falofr] ./ (R * T))))
    Qfalofr = Kf_f .* prod(X .^ order_f, dims=1)' .- Kr_f .* prod(X .^ ν2[:,falofr], dims=1)' .* reversible[falofr]
    ####
    Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
    Kr = Kf ./ (((pa/(R * T)) .^ sum(ν[:,elmr], dims=1)' .* exp.(ΔS[elmr] ./ R - ΔH[elmr] ./ (R * T))))
    Qelmr = Kf .* prod(X .^ order, dims=1)' .- Kr .* prod(X .^ ν2[:,elmr], dims=1)' .* reversible[elmr]
    ##### Computing ω_dot 
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h ./ W - R ./ W .* T   # Internal energy for species
    p = sum(X) * R * T # pressure
    Q = [Qtbd' Qfalofr' Qelmr']
    ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
    ω_dot = W .* sum(ν_new .* Q, dims=2)
    ###### mout_dot computation
#     cp_total = sum(cp ./ W .* Y)
#     _area = 1
#     _gamma_s = cp_total / cv
#     _press = sum(X) * R * T
#     _r_gas_specific = cp_total - cv
#     _p_ratio = 101325 / (_press + 10.0)
#     power_1 = 2.0 / _gamma_s
#     power_2 = (_gamma_s + 1.0) / (_gamma_s)
#     pressure_term = _p_ratio ^ power_1 - _p_ratio ^ power_2
#     #println(_press)
#     #println(_p_ratio)
#     #println(pressure_term)
#     _sqrt_term = 2. * _gamma_s * _r_gas_specific * T / (_gamma_s - 1.0)
#     _sqrt_term *= pressure_term
#     _sqrt_term = (_sqrt_term)^(0.5)

#     _mdot_unchoked = m * _area * _sqrt_term

#     power = (_gamma_s + 1.0) / (_gamma_s - 1.0)
#     _gamma_term = (2. / (_gamma_s + 1.0)) ^ power
#     _sqrt = (_gamma_s * _r_gas_specific * T * _gamma_term)^(0.5)
#     _mdot_choked = m * _area * _sqrt
#     _p_crit_downstream = _press * (2. / (_gamma_s + 1.0)) ^ (_gamma_s / (_gamma_s - 1.0))
#     if (_p_crit_downstream < 101325)
#         mout_dot = _mdot_unchoked
#     else
#         mout_dot = _mdot_choked
#     end
    ###### Species Conservation
    mgen_dot = V .* ω_dot
    min_dot = min
    mout_dot = mout
    m_dot = min_dot - mout_dot
    Y_dot = (1 / m) .* (min_dot .* (Yin .- Y) .+ mgen_dot)
    Y_dot = (1 / m) .* (min_dot .* (Yin .- Y) .- mout_dot .* (Yout .- Y) .+ mgen_dot)
    ###### Energy Conservation
    T_dot = 1 / (m * cv) * (q(t) + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    #T_dot = 1 / (m * cv + n_ * R) * (q(t) + min_dot * (hin - sum(u .* Yin)) .+ mout_dot * sum(u .* (Yout .- Y)) - p * V / m * mout_dot - sum(mgen_dot .* u) - R * T * (m_dot * sum(Y ./ W) + m * sum(Y_dot ./ W)))
    #T_dot = 1 / (m * cv) * (q(t) + min_dot * (hin - sum(u .* Yin)) .+ mout_dot * sum(u .* (Yout .- Y)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    T_dot = 1 / (m * cv) * (q(t) + min_dot * hin - mout_dot * sum(h ./ W .* Yout) - m_dot * sum(u .* Y) - m * sum(u .* Y_dot))
    return [m_dot; T_dot; Y_dot]
end;

In [ ]:
## RK4
#dt = 4e-9
#coeff = 2.232973258446763e-06
W[5] = 44.0098
timestep = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/timestep.npy")
min = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/min_dot.npy") 
mout = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/mout_dot.npy") 
timestep = range(0,stop=624*2.230011e-06,length=624)
n_t = size(timestep)[1]
#n_t = 2000
#T = T_init # Temperature
T = 349.999999994
#Y = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/ini_mass_fraction.npy") # Mass fractions
#Y = [0; 0; 1; 0; 0; 0]
Y = [8.04293023e-04; 0.0; 9.99195707e-01; 0.0; 0.0; 0.0]
mTY = [m_;T;Y]
TYt = zeros(N+1, n_t)
Tt = zeros(n_t)
Pt = zeros(n_t)
Yt = zeros(N,n_t)
mt = zeros(n_t)
mt[1] = m_
Y0 = copy(Y)
Yt[:,1] .= Y
Tt[1] = T
X = (m_ / V) * Y ./ W
Pt[1] = sum(X) * R * T;

In [ ]:
for i = 1:n_t - 1
    dt = timestep[i+1] - timestep[i]
    k1 = f(mTY, timestep[i], min[i], mout[i])
    k2 = f(mTY + 0.5 * dt * k1, timestep[i] + 0.5 * dt, min[i], mout[i])
    k3 = f(mTY + 0.5 * dt * k2, timestep[i] + 0.5 * dt, min[i], mout[i])
    k4 = f(mTY + dt * k3, timestep[i] + dt, min[i + 1], mout[i + 1])
    mTY += 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    #TY += dt * f(TY,timestep[i])
    Tt[i+1] = mTY[2]
    #println(mTY[2])
    #println(i)
    Y = mTY[3:end]
    mt[i + 1] = mTY[1]
    Yt[:,i+1] = Y
    X = (mTY[1] / V) * Y ./ W
    TYt[:,i+1] = mTY[2:end]
    Pt[i+1] = sum(X) * R * mTY[2]
end

In [ ]:
#X = [i * 4e-9 for i = 1:2000]
Tt_cant = py"np.load"("temperature.npy")
plot(timestep, Tt, label = "0D Model")
plot!(timestep, Tt_cant, label = "Torch Case")
xlabel!("Time (s)")
ylabel!("Temperature (K)")
#savefig("Temperature.pdf")

In [ ]:
xo2 = py"np.load"("xo2.npy")
plot((Yt[1,2:end]), label = "0D model")
plot!(xo2, label = "Torch Case")
xlabel!("Time Steps")
ylabel!("Mass Fraction of O2")

In [ ]:
ch4 = py"np.load"("xch4.npy")
plot((Yt[3,2:end]), label = "0D model")
plot!(ch4, label = "Torch Case")
xlabel!("Time Steps")
ylabel!("Mass Fraction of CH4")

In [ ]:
Pt_cant = py"np.load"("pressure.npy")
plot(timestep, Pt, label = "0D Model")
plot!(timestep, Pt_cant, label = "Case 0")
xlabel!("Time")
ylabel!("Pressure")
#savefig("Pressure.pdf")

In [ ]:
Rhot_cant = py"np.load"("data/total_mass.npy")
Rhot_cant = py"np.load"("total_rho.npy")
plot(timestep, mt, label = "0D Model")
plot!(timestep, Rhot_cant,  label = "Torch Case")
xlabel!("Time")
ylabel!("Mass")
#savefig("Mass.pdf")

In [ ]:
# dt = 1e-12
# for i = 1:n_t
#     k1 = f(TY)
#     k2 = f(TY + 0.5 * dt * k1)
#     k3 = f(TY + 0.5 * dt * k2)
#     k4 = f(TY + dt * k3)
#     TY += 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
#     #TY += dt * f(TY)
#     Tt[i] = TY[1]
#     print(TY[1])
#     println(i)
#     Y = TY[2:end]
#     Yt[:,i] = Y
#     X = ρ * Y ./ W
#     p = sum(X) * R * TY[1]
#     TYt[:,i] = TY
#     Pt[i] = p
# end

In [ ]:
function forward!(dty, ty, p, t)
    T = ty[1]
    Y = ty[2:end]
    ρ = m / V # density
    X = ρ * Y ./ W # Concentration
    Q = ones(M) # Individual progress rates
    # cpHS = compute_cpHS(NASA_coeffs, R, T)
    # cp = cpHS[:,1]
    # h = cpHS[:,2]
    # s = cpHS[:,3]
    #islarge = NASA_coeffs[:,1] .< T * ones(N)
    fhi = f_hi(NASA_coeffs, T, R)
    #flo = f_lo(NASA_coeffs, T, R)
    
#     cp = islarge .* fhi[:,1] + (1 .- islarge) .* flo[:,1]
#     h = islarge .* fhi[:,2] + (1 .- islarge) .* flo[:,2]
#     s = islarge .* fhi[:,3] + (1 .- islarge) .* flo[:,3]
    cp = fhi[:,1] 
    h = fhi[:,2] 
    s = fhi[:,3] 
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    ####
    M_t = efficiency_t' * X
    Kf_t = Af_t .* (T .^ β_t) .* exp.(-E_t ./ (R * T)) .* M_t
    Kr_t = Kf_t ./ (((pa/(R * T)) .^ sum(ν[:,tbd], dims=1)' .* exp.(ΔS[tbd] ./ R - ΔH[tbd] ./ (R * T))))
    Qtbd = Kf_t .* prod(X .^ order_t, dims=1)' .- Kr_t .* prod(X .^ ν2[:,tbd], dims=1)' .* reversible[tbd]
    ####
    M_f = efficiency_f' * X
    Kf_lo = Af_lo .* (T .^ β_lo) .* exp.(-E_lo ./ (R * T)) .* M_f
    Kf_hi = Af_hi .* (T .^ β_hi) .* exp.(-E_hi ./ (R * T)) 
    Pr = Kf_lo ./ Kf_hi
    Fac = ones(size(falofr)[1])
    Fac[troefall] = [compute_falloff(T, Pr[s], troefall_coeff[:,i]) for (i,s) in enumerate(troefall)]
    Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
    Kr_f = Kf_f ./ (((pa/(R * T)) .^ sum(ν[:,falofr], dims=1)' .* exp.(ΔS[falofr] ./ R - ΔH[falofr] ./ (R * T))))
    Qfalofr = Kf_f .* prod(X .^ order_f, dims=1)' .- Kr_f .* prod(X .^ ν2[:,falofr], dims=1)' .* reversible[falofr]
    ####
    Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
    Kr = Kf ./ (((pa/(R * T)) .^ sum(ν[:,elmr], dims=1)' .* exp.(ΔS[elmr] ./ R - ΔH[elmr] ./ (R * T))))
    Qelmr = Kf .* prod(X .^ order, dims=1)' .- Kr .* prod(X .^ ν2[:,elmr], dims=1)' .* reversible[elmr]
    ##### Computing ω_dot 
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h ./ W - R ./ W .* T   # Internal energy for species
    p = sum(X) * R * T # pressure
    Q = [Qtbd' Qfalofr' Qelmr']
    ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
    ω_dot = W .* sum(ν_new .* Q, dims=2)
    ###### Species Conservation
    mgen_dot = V .* ω_dot
    dty[2:end] = (1 / m) .* (min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot) 
    ###### Energy Conservation
    dty[1] = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
end;

In [ ]:
# TY = [T;Y]
# u0 = TY
# tspan = (0.0,0.001)
# prob = ODEProblem(forward!,u0,tspan)
# sol = DifferentialEquations.solve(prob)

In [ ]:
# using ModelingToolkit
# de = modelingtoolkitize(prob)